In [1]:
import os 
new_directory = "C:/Users/natha/FinalProjectDS4010A/src"  # Replace with the actual path
os.chdir(new_directory)

In [2]:
from pathlib import Path
import pandas as pd
import geopandas as gpd

import pyarrow.compute as pc
from networkx.algorithms.bipartite.basic import color

from datalake import Datalake

import matplotlib.pyplot as plt

In [3]:
data = Datalake("C:/Users/natha/FinalProjectDS4010A/data")

In [4]:
query = pc.field('year').isin([2018, 2019, 2020])

In [5]:
weather = data.query_ghcnd(query)

In [6]:
hexagon_df = gpd.read_parquet("C:/Users/natha/FinalProjectDS4010A/data/curated/hexagon_data.parquet")
hexagon_df

,Hexagon_ID,geometry
0,832616fffffffff,"POLYGON ((43.44809 -102.28712, 43.92731 -101.7..."
1,83278bfffffffff,"POLYGON ((44.08338 -103.53595, 44.56479 -103.0..."
2,8326a5fffffffff,"POLYGON ((43.03668 -103.6758, 43.52552 -103.16..."
3,832612fffffffff,"POLYGON ((42.39644 -102.4393, 42.88309 -101.92..."
4,832610fffffffff,"POLYGON ((42.79653 -101.05902, 43.27353 -100.5..."
...,...,...
707,832b18fffffffff,"POLYGON ((45.16986 -67.50096, 44.87798 -68.328..."
708,832b1cfffffffff,"POLYGON ((46.0885 -66.58525, 45.80179 -67.4345..."
709,834414fffffffff,"POLYGON ((23.68266 -82.3104, 24.13408 -81.8695..."
710,834416fffffffff,"POLYGON ((23.9852 -81.26917, 24.42962 -80.8296..."


In [7]:
fire = data.query_fire_point()

In [8]:
weather = weather[weather['Hexagon_ID'].isin(hexagon_df['Hexagon_ID'].unique())]
fire = fire[fire['Hexagon_ID'].isin(hexagon_df['Hexagon_ID'].unique())]


In [9]:
fire['DISCOVERYDATE'] = pd.to_datetime(fire['DISCOVERYDATE'])  # Ensure it's in datetime format

fire['year'] = fire['DISCOVERYDATE'].dt.year
fire['month'] = fire['DISCOVERYDATE'].dt.month
fire['day'] = fire['DISCOVERYDATE'].dt.day

In [10]:
import pandas as pd

# Ensure copies to avoid SettingWithCopyWarning
temp2 = weather.copy()
fire2 = fire.copy()

# Convert to datetime
temp2.loc[:, "date"] = pd.to_datetime(temp2[["year", "month", "day"]])
fire2.loc[:, "date"] = pd.to_datetime(fire2["DISCOVERYDATE"], errors="coerce")

# Group by Hexagon_ID and date, and compute mean tmax/tmin across stations for each Hexagon_ID
temp_grouped = temp2.groupby(["Hexagon_ID", "date"], as_index=False).agg({
    "tmax": "mean", 
    "tmin": "mean", 
    "prcp": "mean", 
    "snow": "mean", 
    "awnd": "mean"
})

# Sort by Hexagon_ID and date
temp_grouped = temp_grouped.sort_values(by=["Hexagon_ID", "date"])

# Compute 3-day rolling average within each Hexagon_ID
temp_grouped.loc[:, "tmax_avg"] = temp_grouped.groupby("Hexagon_ID")["tmax"].rolling(window=3, min_periods=2).mean().reset_index(level=0, drop=True)
temp_grouped.loc[:, "tmin_avg"] = temp_grouped.groupby("Hexagon_ID")["tmin"].rolling(window=3, min_periods=2).mean().reset_index(level=0, drop=True)
temp_grouped.loc[:, "prcp_avg"] = temp_grouped.groupby("Hexagon_ID")["prcp"].rolling(window=3, min_periods=2).mean().reset_index(level=0, drop=True)
temp_grouped.loc[:, "snow_avg"] = temp_grouped.groupby("Hexagon_ID")["snow"].rolling(window=3, min_periods=2).mean().reset_index(level=0, drop=True)
temp_grouped.loc[:, "awnd_avg"] = temp_grouped.groupby("Hexagon_ID")["awnd"].rolling(window=3, min_periods=2).mean().reset_index(level=0, drop=True)

# Create binary fire occurrence column and aggregate multiple fires per day per hexagon
fire2.loc[:, "fire_occurred"] = 1  # Indicate fire presence
fire2 = fire2.groupby(["Hexagon_ID", "date"], as_index=False).agg({"fire_occurred": "max"})  # Ensure only one fire_occurred per day per hexagon

# Merge temperature data with fire occurrence data
result = temp_grouped.merge(fire2, on=["Hexagon_ID", "date"], how="left")

# Fill NaN values in 'fire_occurred' with 0 (indicating no fire)
result.loc[:, "fire_occurred"] = result["fire_occurred"].fillna(0).astype(int)

# Extract year, month, day for output
result.loc[:, "year"] = result["date"].dt.year
result.loc[:, "month"] = result["date"].dt.month
result.loc[:, "day"] = result["date"].dt.day

# Select relevant columns
final_df = result[["Hexagon_ID", "year", "month", "day", "tmax_avg", "tmin_avg", "prcp_avg", "snow_avg", "awnd_avg", "fire_occurred", "date"]]

# The resulting dataframe 'final_df' will now contain aggregated data for each Hexagon_ID
print(final_df.tail(100))


             Hexagon_ID  year  month  day    tmax_avg    tmin_avg   prcp_avg  \
767572  8348f6fffffffff  2020      9   23  311.166667  145.500000   0.000000   
767573  8348f6fffffffff  2020      9   24  326.833333  165.000000   0.000000   
767574  8348f6fffffffff  2020      9   25  338.000000  172.333333   0.000000   
767575  8348f6fffffffff  2020      9   26  350.000000  176.000000   0.000000   
767576  8348f6fffffffff  2020      9   27  359.333333  160.166667   0.000000   
...                 ...   ...    ...  ...         ...         ...        ...   
767667  8348f6fffffffff  2020     12   27  175.833333   -7.333333   0.000000   
767668  8348f6fffffffff  2020     12   28  206.333333   16.666667   0.000000   
767669  8348f6fffffffff  2020     12   29  232.166667   45.333333   0.000000   
767670  8348f6fffffffff  2020     12   30  225.666667   52.833333   3.733333   
767671  8348f6fffffffff  2020     12   31  174.000000   32.333333  53.466667   

         snow_avg   awnd_avg  fire_occu

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

#invalidHexes = {'832b92fffffffff', '8312f2fffffffff', '832705fffffffff', '831354fffffffff', '8312e0fffffffff', '834473fffffffff', '8348f0fffffffff'}
#final_df = final_df[~final_df["Hexagon_ID"].isin(invalidHexes)]

#Training data here
weather = final_df[(final_df['date'] >= '2018-01-01') & (final_df['date'] <= '2019-12-31')]

# Filter weatherTest dataset for 2020 (test data)
weatherTest = final_df[(final_df['date'] >= '2020-01-01') & (final_df['date'] <= '2020-12-31')]

# Preprocessing the training data (weather)
weather_cleaned = weather.dropna(subset=["Hexagon_ID"])  # Drop rows with missing critical columns
weather_cleaned = weather_cleaned.copy()  # Ensure you're working with a new copy

# Preprocessing the testing data (weatherTest)
weatherTest_cleaned = weatherTest.dropna(subset=["Hexagon_ID"])  # Drop rows with missing critical columns
weatherTest_cleaned = weatherTest_cleaned.copy()  # Ensure you're working with a new copy

# One hot encode 'Hexagon_ID' and 'month' for weather
le = LabelEncoder()
le.fit(pd.concat([weather_cleaned['Hexagon_ID'], weatherTest_cleaned['Hexagon_ID']]))

le2 = LabelEncoder()
le2.fit(pd.concat([weather_cleaned['month'], weatherTest_cleaned['month']]))
weather_cleaned["Hexagon_ID_encoded"] = le.transform(weather_cleaned["Hexagon_ID"])  # Encode Hexagon_ID
weather_cleaned["month_encoded"] = le2.transform(weather_cleaned["month"])  # Encode month

# Fill missing values with mean for relevant columns
mean_tmax = weather_cleaned["tmax_avg"].mean()
weather_cleaned["tmax_avg"] = weather_cleaned["tmax_avg"].fillna(mean_tmax)

mean_tmin = weather_cleaned["tmin_avg"].mean()
weather_cleaned["tmin_avg"] = weather_cleaned["tmin_avg"].fillna(mean_tmin)

mean_awnd = weather_cleaned["awnd_avg"].mean()
weather_cleaned["awnd_avg"] = weather_cleaned["awnd_avg"].fillna(mean_awnd)

mean_prcp = weather_cleaned["prcp_avg"].mean()
weather_cleaned["prcp_avg"] = weather_cleaned["prcp_avg"].fillna(mean_prcp)

mean_snow = weather_cleaned["snow_avg"].mean()
weather_cleaned["snow_avg"] = weather_cleaned["snow_avg"].fillna(mean_snow)


# Use the same label encoder for 'Hexagon_ID' and 'month' as in training data
weatherTest_cleaned["Hexagon_ID_encoded"] = le.transform(weatherTest_cleaned["Hexagon_ID"])  # Use the same encoder
weatherTest_cleaned["month_encoded"] = le2.transform(weatherTest_cleaned["month"])  # Use the same encoder for month

# Fill missing values in weatherTest_cleaned with the mean of each column
mean_tmax = weatherTest_cleaned["tmax_avg"].mean()
weatherTest_cleaned["tmax_avg"] = weatherTest_cleaned["tmax_avg"].fillna(mean_tmax)

mean_tmin = weatherTest_cleaned["tmin_avg"].mean()
weatherTest_cleaned["tmin_avg"] = weatherTest_cleaned["tmin_avg"].fillna(mean_tmin)

mean_awnd = weatherTest_cleaned["awnd_avg"].mean()
weatherTest_cleaned["awnd_avg"] = weatherTest_cleaned["awnd_avg"].fillna(mean_awnd)

mean_prcp = weatherTest_cleaned["prcp_avg"].mean()
weatherTest_cleaned["prcp_avg"] = weatherTest_cleaned["prcp_avg"].fillna(mean_prcp)

mean_snow = weatherTest_cleaned["snow_avg"].mean()
weatherTest_cleaned["snow_avg"] = weatherTest_cleaned["snow_avg"].fillna(mean_snow)


X_train = weather_cleaned[["Hexagon_ID_encoded", "tmax_avg", "tmin_avg", "prcp_avg", "snow_avg", "awnd_avg", "month_encoded"]]
y_train = weather_cleaned["fire_occurred"]  # Target variable for training

X_test = weatherTest_cleaned[["Hexagon_ID_encoded", "tmax_avg", "tmin_avg", "prcp_avg", "snow_avg", "awnd_avg", "month_encoded"]]
y_test = weatherTest_cleaned["fire_occurred"]  # Target variable for testing

# Initialize the RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model to the training data
rf_model.fit(X_train, y_train)

# Get predicted probabilities for fire occurrence (class 1)
probs = rf_model.predict_proba(X_test)[:, 1]  # Probability of fire occurring

In [12]:
# Set custom threshold (lower than default 0.5)
new_threshold = 0.2
y_pred_adjusted = (probs > new_threshold).astype(int)

# Evaluate the model's performance
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_adjusted))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_adjusted))

# Optional: Feature importance (if you'd like to see which features contribute the most)
print("\nFeature Importance:")
print(rf_model.feature_importances_)

# Print the predicted probabilities (optional, to see the output probabilities)
print("\nPredicted Probabilities for Test Data:")
print(probs)

Confusion Matrix:
[[249919   2185]
 [  3182    879]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99    252104
         1.0       0.29      0.22      0.25      4061

    accuracy                           0.98    256165
   macro avg       0.64      0.60      0.62    256165
weighted avg       0.98      0.98      0.98    256165


Feature Importance:
[0.17916652 0.23041774 0.23111003 0.16039211 0.0105356  0.14032864
 0.04804937]

Predicted Probabilities for Test Data:
[0.   0.01 0.01 ... 0.   0.   0.  ]


In [13]:
#Adding in predicted probabilities
weatherTest.loc[:, "predicted_probabilities"] = probs
weatherTest

#Merging the data
merged_data = weatherTest.merge(hexagon_df, on="Hexagon_ID", how="inner")  # Use "left" if keeping all hexagons
merged_data

C:\Users\natha\AppData\Local\Temp\ipykernel_76264\1852380507.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weatherTest.loc[:, "predicted_probabilities"] = probs


,Hexagon_ID,year,month,day,tmax_avg,tmin_avg,prcp_avg,snow_avg,awnd_avg,fire_occurred,date,predicted_probabilities,geometry
0,8312c9fffffffff,2020,1,1,46.603175,-55.865079,1.595238,0.000000,96.666667,0.0,2020-01-01,0.00,"POLYGON ((49.58621 -112.59069, 49.05271 -112.9..."
1,8312c9fffffffff,2020,1,2,43.722222,-48.484127,4.119048,0.000000,97.000000,0.0,2020-01-02,0.01,"POLYGON ((49.58621 -112.59069, 49.05271 -112.9..."
2,8312c9fffffffff,2020,1,3,42.285714,-67.428571,4.577381,6.000000,76.000000,0.0,2020-01-03,0.01,"POLYGON ((49.58621 -112.59069, 49.05271 -112.9..."
3,8312c9fffffffff,2020,1,4,45.809524,-74.809524,4.410714,6.000000,87.333333,0.0,2020-01-04,0.01,"POLYGON ((49.58621 -112.59069, 49.05271 -112.9..."
4,8312c9fffffffff,2020,1,5,51.190476,-59.333333,0.458333,6.000000,95.333333,0.0,2020-01-05,0.00,"POLYGON ((49.58621 -112.59069, 49.05271 -112.9..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256160,8348f6fffffffff,2020,12,27,175.833333,-7.333333,0.000000,0.000000,21.333333,0.0,2020-12-27,0.02,"POLYGON ((28.93394 -104.03903, 29.48455 -103.5..."
256161,8348f6fffffffff,2020,12,28,206.333333,16.666667,0.000000,0.000000,22.333333,0.0,2020-12-28,0.01,"POLYGON ((28.93394 -104.03903, 29.48455 -103.5..."
256162,8348f6fffffffff,2020,12,29,232.166667,45.333333,0.000000,0.000000,23.333333,0.0,2020-12-29,0.00,"POLYGON ((28.93394 -104.03903, 29.48455 -103.5..."
256163,8348f6fffffffff,2020,12,30,225.666667,52.833333,3.733333,0.000000,29.333333,0.0,2020-12-30,0.00,"POLYGON ((28.93394 -104.03903, 29.48455 -103.5..."


In [14]:
merged_data

,Hexagon_ID,year,month,day,tmax_avg,tmin_avg,prcp_avg,snow_avg,awnd_avg,fire_occurred,date,predicted_probabilities,geometry
0,8312c9fffffffff,2020,1,1,46.603175,-55.865079,1.595238,0.000000,96.666667,0.0,2020-01-01,0.00,"POLYGON ((49.58621 -112.59069, 49.05271 -112.9..."
1,8312c9fffffffff,2020,1,2,43.722222,-48.484127,4.119048,0.000000,97.000000,0.0,2020-01-02,0.01,"POLYGON ((49.58621 -112.59069, 49.05271 -112.9..."
2,8312c9fffffffff,2020,1,3,42.285714,-67.428571,4.577381,6.000000,76.000000,0.0,2020-01-03,0.01,"POLYGON ((49.58621 -112.59069, 49.05271 -112.9..."
3,8312c9fffffffff,2020,1,4,45.809524,-74.809524,4.410714,6.000000,87.333333,0.0,2020-01-04,0.01,"POLYGON ((49.58621 -112.59069, 49.05271 -112.9..."
4,8312c9fffffffff,2020,1,5,51.190476,-59.333333,0.458333,6.000000,95.333333,0.0,2020-01-05,0.00,"POLYGON ((49.58621 -112.59069, 49.05271 -112.9..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256160,8348f6fffffffff,2020,12,27,175.833333,-7.333333,0.000000,0.000000,21.333333,0.0,2020-12-27,0.02,"POLYGON ((28.93394 -104.03903, 29.48455 -103.5..."
256161,8348f6fffffffff,2020,12,28,206.333333,16.666667,0.000000,0.000000,22.333333,0.0,2020-12-28,0.01,"POLYGON ((28.93394 -104.03903, 29.48455 -103.5..."
256162,8348f6fffffffff,2020,12,29,232.166667,45.333333,0.000000,0.000000,23.333333,0.0,2020-12-29,0.00,"POLYGON ((28.93394 -104.03903, 29.48455 -103.5..."
256163,8348f6fffffffff,2020,12,30,225.666667,52.833333,3.733333,0.000000,29.333333,0.0,2020-12-30,0.00,"POLYGON ((28.93394 -104.03903, 29.48455 -103.5..."


In [39]:
import folium
import shapely.geometry
import matplotlib.colors as mcolors
import numpy as np

one_date = merged_data[merged_data["date"] == '2020-07-05'].copy()  # Make a copy of the DataFrame

# Normalize with 0.5 as the mean
min_val = one_date['predicted_probabilities'].min()
max_val = one_date['predicted_probabilities'].max()

one_date.loc[:, 'normalized_probabilities'] = (one_date['predicted_probabilities'] - min_val) / (max_val - min_val)

# Adjust to ensure mean is 0.5
mean_shift = 0.5 - one_date['normalized_probabilities'].mean()
one_date.loc[:, 'normalized_probabilities'] += mean_shift

# Clip to ensure values remain in [0,1]
one_date.loc[:, 'normalized_probabilities'] = np.clip(one_date['normalized_probabilities'], 0, 1)

# Define a colormap from green (low) to red (high)
cmap = mcolors.LinearSegmentedColormap.from_list("green_red", ["green", "yellow", "red"])

# Create a folium map
m = folium.Map(location=[39.8, -98.5], zoom_start=4)

# Normalize probabilities to map to colors
norm = mcolors.Normalize(vmin=one_date['normalized_probabilities'].min(), 
                         vmax=one_date['normalized_probabilities'].max())

# Add each hexagon polygon to the map
for _, row in one_date.iterrows():
    # Extract coordinates from the polygon geometry
    polygon = row['geometry']
    coordinates = [(lon, lat) for lon, lat in polygon.exterior.coords]

    # Get color based on normalized probability
    color = mcolors.to_hex(cmap(norm(row['normalized_probabilities'])))

    # Add the polygon to the map
    folium.Polygon(
        locations=coordinates,
        color=color,  # Outline color
        weight=1,
        opacity=0.1,  
        fill=True,
        fill_color=color,  # Fill color based on probability
        fill_opacity=0.6
    ).add_to(m)

# Show the map
m
